# **Chapter 5: Formatting Output & Speaking for Claude**

---
**Lesson:**

Claude can format its output in a wide variety of ways. You just need to ask for it to do so!
One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also use XML tags to make Claude's output clearer and more easily understandable to humans.

---

First we will setup our dependencies

In [2]:
%%capture
#Cell [1]

#langchain
%pip install langchain

#Install libraries 
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries
import json
import os
import sys
import boto3
import botocore

#Set up Bedrock runtime client
bedrock_runtime = boto3.client('bedrock-runtime')
boto3_bedrock = boto3.client('bedrock')

#Set up ModelID (Claude 3 Sonnet in this case)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

In [3]:
#Cell [2]

#Setup Model invocation function 
#Code modified from documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_Claude3_Text_section.html
def get_response(prompt_data):
    response = bedrock_runtime.invoke_model(
                    modelId=modelId,
                    body=json.dumps(
                        {
                            "anthropic_version": "bedrock-2023-05-31",
                            "max_tokens": 1024,
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [{"type": "text", "text": prompt_data}],
                                }
                            ],
                        }
                    ),
                )
    result = json.loads(response.get("body").read())
    response = result.get("content", [])[0]['text']
    return response

# **Examples:**

**Example 5.1 - Tagging**

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by telling Claude to put the poem in XML tags.

In [5]:
#Create a prompt template that has input variable(s)
from langchain import PromptTemplate
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Rabbit")

response = get_response(prompt)

print(response)

<haiku>
Soft fur, twitching nose,
Nibbling clover in the field,
Rabbit, nature's joy.
</haiku>


**Example 5.2 - Half Tagging**

Why is this something we'd want to do? Well, having the output in XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags.		

An extension of this technique is to put the first XML tag AFTER "Assistant:". When you put text after "Assistant:", you're basically telling Claude that Claude has already said something, and that it should continue from that point onward.
Below, we've done this with the first "haiku" XML tag.		

In [6]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags

Assistant: <haiku>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Cat")

response = get_response(prompt)

print(response)

<haiku>
Feline grace glides by,
Whiskers twitching, eyes aglow,
Purr-fect companion.
</haiku>


**Example 5.3 - JSON**

Claude also excels at using other output formatting styles, notably JSON.

In [7]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Use JSON format with the keys as "first_line", "second_line", and "third_line"

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Cat")

response = get_response(prompt)

print(response)

{
  "first_line": "Feline grace and poise,",
  "second_line": "Purring softly in the sun,",
  "third_line": "Cat's quiet wisdom."
}


**Example 5.4 - Combination**

Below is an example of multiple input variables in the same prompt AND output formatting specification, all done using XML tags.		

In [8]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL","ADJECTIVE"], 
    template="""

Human: Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE} email> XML tags.

Assistant: <{ADJECTIVE}_email>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.", ADJECTIVE="Olde English")

response = get_response(prompt)

print(response)

<Olde English_email>
Hail Zack, good sir! I entreat thee for a swift recounting upon the prompt thou wert tasked to indite. Let not more time be squandered, and bring word anon of thy progress on this noble endeavor.
</Olde English_email>


# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 5.1 - Steph Curry GOAT**

Forced to make a choice, Claude sometimes designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?
Modify the "best basketball player" prompt below and use the "speaking for Claude" technique to make Claude make a detailed argument that the best basketball player of all time is Stephen Curry.

In [9]:
response = get_response("""

Human: Who is the overall best basketball player of all time? Make the case for one player.

Assistant:""")

print(response)

There is no definitive consensus on the best basketball player of all time, as it's a subjective matter that depends on how one weighs various factors. However, many experts and fans make a strong case for Michael Jordan being the greatest ever. Here's why:

1. Unparalleled Success and Accolades:
- 6-time NBA champion (1991, 1992, 1993, 1996, 1997, 1998)
- 5-time NBA MVP (1988, 1991, 1992, 1996, 1998)
- 10-time NBA scoring champion
- Defensive Player of the Year (1988)
- 14-time NBA All-Star

2. Dominance and Impact on the Game:
- His Airness revolutionized the game with his athleticism, skill, and competitive spirit.
- He was a transcendent figure who made the NBA a global phenomenon.
- His incredible scoring ability, clutch performances, and will to win set him apart.

3. Iconic Moments and Legacy:
- The Flu Game, The Shrug, The Last Shot, and many other legendary performances.
- His impact on basketball culture, merchandise, and branding is unmatched.
- He is arguably the most famou

**Exercise 5.2 - Two Haikus**

Modify the haiku prompt in the highlighted template box below and use XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [10]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags

Assistant: <haiku>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL="Cat")

response = get_response(prompt)

print(response)

<haiku>
Soft paws, graceful steps,
Purring companion, Cat,
Ruler of the home.
</haiku>


**Exercise 5.3 - Two Haikus, Two Animals**

Modify the haiku prompt in the cell below so that Claude produces two haikus about two different animals. 
Use {ANIMAL1} as a stand-in for the first substitution, and {ANIMAL2} as a stand-in for the second substitution.

In [ ]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], #Change Input variables 
    template="""

Human: Please write a haiku about {ANIMAL}. Put it in <haiku> tags 

Assistant: <haiku>"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(ANIMAL1="Cat",ANIMAL2="Dog") #We want these to be inputs to the prompt

response = get_response(prompt)

print(response)

# Chapter 5 - END.